# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

price_data_path = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(price_data_path, "**/*.parquet"), recursive=True)
print(f'Found {len(parquet_files)} parquet files in {price_data_path}' )


Found 2991 parquet files in ../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [8]:

dd_px = dd.read_parquet(parquet_files)
# Define a function to compute features per ticker
def add_features(df):
    # Ensure data is sorted by Date
    df = df.sort_values('Date')
    # Lagged variables
    df['Close_lag_1'] = df['Close'].shift(1)
    df['Adj_Close_lag_1'] = df['Adj Close'].shift(1)
    # Returns
    df['returns'] = (df['Close'] / df['Close_lag_1']) - 1
    # High-low range
    df['hi_lo_range'] = df['High'] - df['Low']
    return df

# Apply transformation per ticker
dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    add_features,
    meta={
        'Date': 'datetime64[ns]',
        'Open': 'f8',
        'High': 'f8',
        'Low': 'f8',
        'Close': 'f8',
        'Adj Close': 'f8',
        'Volume': 'i8',
        'source': 'object',
        'ticker': 'object',
        'Year': 'i4',
        'Close_lag_1': 'f8',
        'Adj_Close_lag_1': 'f8',
        'returns': 'f8',
        'hi_lo_range': 'f8'
    }
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
import pandas as pd
# Compute and convert to Pandas
df_pandas = dd_feat.compute()
print(df_pandas.head())
#Sort Data by ticker and Date
df_pandas = df_pandas.sort_values(['ticker', 'Date']).reset_index(drop=True)
print(df_pandas.head())

#Calculate the 10 day rolling mean of the returns for each ticker
df_pandas['returns_10d_ma'] = (
    df_pandas.groupby('ticker')['returns']
    .transform(lambda x: x.rolling(window=10, min_periods=1).mean())
)

# Inspect the result to verify the rolling average
print(df_pandas[['ticker', 'Date', 'returns', 'returns_10d_ma']].head(15))

/Users/seanbrennan/production-dsi/.venv/lib/python3.11/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/Users/seanbrennan/production-dsi/.venv/lib/python3.11/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/Users/seanbrennan/production-dsi/.venv/lib/python3.

        Date  Open   High    Low  Close  Adj Close  Volume   source ticker  \
0 1980-05-06   0.0  2.250  2.125  2.125   0.463949   300.0  WEN.csv    WEN   
1 1980-05-07   0.0  2.375  2.250  2.250   0.491240  2000.0  WEN.csv    WEN   
2 1980-05-08   0.0  2.375  2.250  2.375   0.518531  1700.0  WEN.csv    WEN   
3 1980-05-09   0.0  2.375  2.250  2.375   0.518531   700.0  WEN.csv    WEN   
4 1980-05-12   0.0  2.375  2.250  2.250   0.491240  1800.0  WEN.csv    WEN   

   Year  Close_lag_1  Adj_Close_lag_1   returns  hi_lo_range  
0  1980          NaN              NaN       NaN        0.125  
1  1980        2.125         0.463949  0.058824        0.125  
2  1980        2.250         0.491240  0.055556        0.125  
3  1980        2.375         0.518531  0.000000        0.125  
4  1980        2.375         0.518531 -0.052632        0.125  
        Date       Open       High        Low      Close  Adj Close  \
0 1999-11-18  32.546494  35.765381  28.612303  31.473534  27.068665   
1 1999-11-1

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
It isn't necessary. IT could have worked in dask as well using .rolling().   
+ Would it have been better to do it in Dask? Why?
It seems the answer is no. In class it was mentionned to try Pandas first by reducing teh data using aggregation, then use pandas... Now that the data is processed, it can fit into my computer's ram and pandas is faster than dask at this point (?).

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.